In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rivabar as rb

%matplotlib qt

## Branco River, Brazil - single Landsat TIF file

* link to data file: https://www.dropbox.com/s/aa7eq8ky0y1wryl/LC08_232060_20140219.tif?dl=0

In [4]:
dirname = '/Users/zoltan/Dropbox/Channels/Fluvial/Branco/'
fname = 'LC08_232060_20140219.tif'
start_x = 675796.2
start_y = 98338.8
end_x = 628190.3
end_y = -91886.6

mndwi, G, G_primal, G_rook, gdf2, left_utm_x, right_utm_x, lower_utm_y, upper_utm_y =\
                rb.extract_centerline(fname, dirname, start_x, start_y, end_x, end_y,\
                file_type='single_tif', ratio = 20,\
                ch_belt_smooth_factor=1e8, n_pixels_per_channel = 5, remove_smaller_components=True)

fig = rb.plot_im_and_lines(mndwi, left_utm_x, right_utm_x, lower_utm_y, upper_utm_y, 
                            G_rook, G_primal, smoothing=False)

removing small holes
running skeletonization
building graph from skeleton
finding reasonable starting and ending points on graph edges
start_ind: 707
end_ind: 708
finding nodes that are within a certain radius of the path


100%|████████████████████████████████████████| 214/214 [00:01<00:00, 131.70it/s]


creating list of linestrings for polygonization


100%|████████████████████████████████████████| 422/422 [00:00<00:00, 442.65it/s]
/Users/zoltan/anaconda3/envs/channelmapper/lib/python3.10/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 22 disconnected components.
 There are 12 islands with ids: 10, 16, 17, 37, 38, 39, 40, 41, 42, 43, 44, 47.
  warnings.warn(message)


first point coords:
4002.0 565.0
second point coords:
2415.0 6907.0
creating centerline polygons


...

100%|████████████████████████████████████| 7396/7396 [00:00<00:00, 31559.73it/s]


getting bank coordinates for the rest of the islands


100%|███████████████████████████████████████████| 64/64 [00:08<00:00,  7.87it/s]


setting half channel widths


100%|███████████████████████████████████████| 188/188 [00:00<00:00, 3680.43it/s]


In [ ]:
# save results to shapefiles:
rb.save_shapefiles(dirname, fname, G_rook, gdf2, fname_add_on='')

## Mamore River, Bolivia - Landsat bands

* link to data folder: https://www.dropbox.com/scl/fo/k7lhzey6kv6kjux92or5d/h?rlkey=zx8g66sprd6038i5sw5xkixgi&dl=0

In [5]:
dirname = '/Users/zoltan/Dropbox/Channels/Fluvial/Bolivia/Mamore/'
fname =  'LC08_L1TP_232071_20160616_20170324_01_T1'
start_x = 303810
start_y = -1851620
end_x = 285603
end_y = -1666131

mndwi, G, G_primal, G_rook, gdf2, left_utm_x, right_utm_x, lower_utm_y, upper_utm_y =\
                rb.extract_centerline(fname, dirname, start_x, start_y, end_x, end_y,\
                file_type='multiple_tifs', ratio = 10,\
                ch_belt_smooth_factor=1e8, n_pixels_per_channel = 2, 
                remove_smaller_components=False, delete_pixels_polys = False)

fig = rb.plot_im_and_lines(mndwi, left_utm_x, right_utm_x, lower_utm_y, upper_utm_y, 
                            G_rook, G_primal, smoothing=False)

removing small holes
running skeletonization
building graph from skeleton
finding reasonable starting and ending points on graph edges
no path between start_ind and end_ind
start_ind: 49094
end_ind: 49095
finding nodes that are within a certain radius of the path


100%|██████████████████████████████████████████| 70/70 [00:00<00:00, 132.90it/s]


creating list of linestrings for polygonization


100%|████████████████████████████████████████| 269/269 [00:00<00:00, 429.72it/s]
/Users/zoltan/anaconda3/envs/channelmapper/lib/python3.10/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 14 disconnected components.
 There are 14 islands with ids: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13.
  warnings.warn(message)


first point coords:
4075.0 6937.0
second point coords:
3470.0 737.0
creating centerline polygons


...

100%|██████████████████████████████████| 14024/14024 [00:00<00:00, 40936.90it/s]


getting bank coordinates for the rest of the islands


100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  8.11it/s]


setting half channel widths


100%|█████████████████████████████████████████| 31/31 [00:00<00:00, 1070.51it/s]


## Brahmaputra River, India / Bangladesh - Landsat bands

* link to data folder: https://www.dropbox.com/scl/fo/wqb8z30wgor3vwg19bdeq/h?rlkey=pq0vxqabhxwnf2rxs07sjk8o1&dl=0

In [7]:
dirname = '/Users/zoltan/Dropbox/Channels/Fluvial/Brahmaputra_2/'
fname = 'LC08_L1TP_138042_20190114_20190131_01_T1'
start_x = 859250.5
start_y = 2903119.2
end_x = 767358.1
end_y = 2776067.9

mndwi, G, G_primal, G_rook, gdf2, left_utm_x, right_utm_x, lower_utm_y, upper_utm_y =\
                rb.extract_centerline(fname, dirname, start_x, start_y, end_x, end_y,\
                file_type='multiple_tifs', ratio = 20,\
                ch_belt_smooth_factor=1e8, n_pixels_per_channel = 40, remove_smaller_components=True, 
                                      mndwi_threshold = 0.01)

fig = rb.plot_im_and_lines(mndwi, left_utm_x, right_utm_x, lower_utm_y, upper_utm_y, 
                            G_rook, G_primal, smoothing=False)

removing small holes
running skeletonization
building graph from skeleton
finding reasonable starting and ending points on graph edges
start_ind: 5040
end_ind: 5041
finding nodes that are within a certain radius of the path


100%|█████████████████████████████████████████| 221/221 [00:18<00:00, 11.77it/s]


creating list of linestrings for polygonization


100%|███████████████████████████████████████| 2820/2820 [00:40<00:00, 69.07it/s]
/Users/zoltan/anaconda3/envs/channelmapper/lib/python3.10/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 95 disconnected components.
 There are 65 islands with ids: 0, 1, 2, 3, 4, 8, 18, 21, 22, 23, 24, 25, 26, 29, 35, 36, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 51, 55, 67, 68, 81, 87, 99, 100, 104, 111, 144, 178, 186, 201, 206, 209, 214, 215, 225, 227, 234, 237, 240, 241, 244, 257, 293, 295, 311, 320, 346, 354, 373, 381, 388, 389, 390, 454, 455.
  warnings.warn(message)


first point coords:
7168.0 2961.0
second point coords:
4094.0 7207.0
creating centerline polygons


...

100%|████████████████████████████████████| 6828/6828 [00:00<00:00, 28466.12it/s]


getting bank coordinates for the rest of the islands


100%|█████████████████████████████████████████| 297/297 [00:39<00:00,  7.54it/s]


setting half channel widths


100%|███████████████████████████████████████| 869/869 [00:00<00:00, 4555.89it/s]


## Purus River, MNDWI image generated in Google Earth Engine

* output from GEE needs to be converted to UTM coordinates before running rivabar
* link to data file: https://www.dropbox.com/scl/fi/7bpz3b4eikzdu59cv4bqu/L8_mndwi_Purus_2017_UTM.tif?rlkey=dkv5ykf1xca0jh478yhn6a6im&dl=0

In [8]:


dirname = "/Users/zoltan/Dropbox/Channels/Fluvial/Purus/"
fname = "L8_mndwi_Purus_2017_UTM.tif"
start_x = 363166.2
start_y = -1011937.1
end_x = 1254178.3
end_y = -528280.7

mndwi, G, G_primal, G_rook, gdf2, left_utm_x, right_utm_x, lower_utm_y, upper_utm_y = \
                rb.extract_centerline(fname, dirname, start_x, start_y, end_x, end_y,\
                file_type='water_index', ratio = 20, mndwi_threshold=0.1, \
                ch_belt_smooth_factor=1e8, n_pixels_per_channel = 40, remove_smaller_components=True,\
                delete_pixels_polys = False, small_hole_threshold = 100)

fig = rb.plot_im_and_lines(mndwi, left_utm_x, right_utm_x, lower_utm_y, upper_utm_y, 
                            G_rook, G_primal, smoothing=False)

removing small holes
running skeletonization
building graph from skeleton
finding reasonable starting and ending points on graph edges
start_ind: 1930
end_ind: 1931
finding nodes that are within a certain radius of the path


100%|█████████████████████████████████████████| 266/266 [00:06<00:00, 40.95it/s]


creating list of linestrings for polygonization


100%|████████████████████████████████████████| 605/605 [00:03<00:00, 163.05it/s]
/Users/zoltan/anaconda3/envs/channelmapper/lib/python3.10/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 17 disconnected components.
 There are 14 islands with ids: 0, 1, 2, 3, 6, 7, 8, 9, 12, 13, 14, 19, 20, 21.
  warnings.warn(message)


first point coords:
98.0 16722.0
second point coords:
29969.0 507.0
creating centerline polygons


...

100%|██████████████████████████████████| 85041/85041 [00:02<00:00, 35343.92it/s]


getting bank coordinates for the rest of the islands


100%|███████████████████████████████████████████| 10/10 [00:16<00:00,  1.61s/it]


setting half channel widths


100%|██████████████████████████████████████████| 31/31 [00:00<00:00, 213.80it/s]


## Adelaide River, Australia, MNDWI generated from Planet Labs data

* link to data file: https://www.dropbox.com/scl/fi/mg84hoc7xls7xqmsbc1f2/ndwi_mosaic.tif?rlkey=ygy5tpyy964gnirvtkyzyzcfg&dl=0

In [9]:
dirname = '/Users/zoltan/Dropbox/Channels/Fluvial/Adelaide/Adelaide_River_1_psscene_analytic_sr_udm2/files/'
fname = 'ndwi_mosaic.tif'
start_x = 753443.4
start_y = 8598434.2
end_x = 743089.7
end_y = 8648110.2

mndwi, G, G_primal, G_rook, gdf2, left_utm_x, right_utm_x, lower_utm_y, upper_utm_y =\
                rb.extract_centerline(fname, dirname, start_x, start_y, end_x, end_y,\
                file_type='water_index', ratio = 20, mndwi_threshold = 0.5, \
                ch_belt_smooth_factor=1e8, n_pixels_per_channel = 100, remove_smaller_components=False)

fig = rb.plot_im_and_lines(mndwi, left_utm_x, right_utm_x, lower_utm_y, upper_utm_y, 
                            G_rook, G_primal, smoothing=False)

removing small holes
running skeletonization
building graph from skeleton
finding reasonable starting and ending points on graph edges
start_ind: 26488
end_ind: 26489
finding nodes that are within a certain radius of the path


100%|██████████████████████████████████████████| 62/62 [00:00<00:00, 169.29it/s]


creating list of linestrings for polygonization


100%|████████████████████████████████████████| 217/217 [00:00<00:00, 440.90it/s]


first point coords:
3622.0 16966.0
second point coords:
257.0 453.0
creating centerline polygons


...

100%|███████████████████████████████████| 29804/29804 [00:06<00:00, 4669.55it/s]


getting bank coordinates for the rest of the islands


0it [00:00, ?it/s]


setting half channel widths


100%|█████████████████████████████████████████████| 2/2 [00:13<00:00,  6.72s/it]
0it [00:00, ?it/s]
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 20.96it/s]
